# Rotational cooling OBE solver
Using the script in "OBE_integrator.py" to simulate rotational cooling in CeNTREX with a single laser driving the rotational cooling transition from $|X, J = 2\rangle$ to $|B, J =1, F_1 = 3/2, F = 1\rangle$ and microwaves coupling $J = 1\leftrightarrow2$ and $J = 2\leftrightarrow3$. This notebook demonstrates how to setup the system to be simulated.

In [ ]:
#Import packages
%load_ext autoreload
%autoreload 2
import sys
import numpy as np
sys.path.append('./molecular-state-classes-and-functions/')
from classes import UncoupledBasisState, CoupledBasisState, State
from functions import ni_range
from OBE_functions import calculate_power_needed, multipassed_laser_E_field

import matplotlib.pyplot as plt
%matplotlib notebook

#Import classes for defining laser and microwave fields
from OBE_classes import OpticalField, MicrowaveField

#Import the OBE integrator script
from OBE_integrator import OBE_integrator

from functools import partial

## X-state
Define what states from the $X ^1\Sigma$ electronic state are to be included in the simulation. Here using all states within J = 0-3

In [ ]:
#Define what states are to be included in the simulation
Js_g = [0,1,2,3] # J values to be included
I_F = 1/2 #Fluorine nuclear spin
I_Tl = 1/2 #Thallium nuclear spin

#Generate a list of approximate ground states. The exact ground states are determined within the main
#simulation function
ground_states_approx = [1*CoupledBasisState(F,mF,F1,J,I_F,I_Tl, electronic_state='X', P = (-1)**J, Omega = 0)
                  for J  in Js_g
                  for F1 in ni_range(np.abs(J-I_F),J+I_F+1)
                  for F in ni_range(np.abs(F1-I_Tl),F1+I_Tl+1)
                  for mF in ni_range(-F, F+1)
                 ]

## B-state
Define what states from the $B ^3\Pi_1$ electronic state are to be included in the simulation. Only need the states that are coupled to J = 1 in the X-state by the laser.

In [ ]:
#Define natural linewidth of excited state
Gamma = 2*np.pi*1.6e6

In [ ]:
#Define what states are to be included in the simulation
J = 1
F1 = 3/2
F = 1
#Generate a list of excited states. The exact excited states are determined within the main
#simulation function
excited_states_approx = [1*CoupledBasisState(F,mF,F1,J,I_F,I_Tl, electronic_state='B', P = -1, Omega = 1)
                  for mF in ni_range(-F, F+1)
                 ]

## Laser field
This section defines the laser field that is driving rotational cooling. 

Goes as follows:
- First define the ground and excited states that are connected by the laser
- Define polarization of laser as function of time. The polarization also includes phase modulation if that's needed.
- Define spatial profile of laser beam
- Define a LaserField object that is eventually passed to the 

In [ ]:
#Define ground states for laser driven transition
Js = [2]
ground_states_laser_approx =  [1*CoupledBasisState(F,mF,F1,J,I_F,I_Tl, electronic_state='X', P = (-1)**J, Omega = 0)
                                  for J  in Js
                                  for F1 in ni_range(np.abs(J-I_F),J+I_F+1)
                                  for F in ni_range(np.abs(F1-I_Tl),F1+I_Tl+1)
                                  for mF in ni_range(-F, F+1)
                                 ]

#Define excited states for laser
excited_states_laser_approx = excited_states_approx

In [ ]:
#Define the "main" states. These are used to calculate the detunings and Rabi rates for the transitions
ground_main_approx = 1*CoupledBasisState(J=2,F1=5/2,F=2,mF=0,I1=1/2,I2=1/2,electronic_state='X', P = 1, Omega = 0)
excited_main_approx = 1*CoupledBasisState(J = 1,F1=3/2,F=1,mF=0,I1=1/2,I2=1/2, electronic_state='B', P = -1, Omega = 1)

In [ ]:
#Polarization as function of time. Also includes phase modulation if that's desired
omega_p = .25*Gamma #Frequency for polarization switching [2pi*Hz]
beta = 0 #Modulation depth for phase modulation
omega_sb = 2*np.pi*1.6e6 #Modulation frequency

#Define polarization as function of time
p_t = lambda omega_p, beta, omega_sb,t: np.array([np.sin(omega_p*t), 0, np.cos(omega_p*t)]) * np.exp(1j*beta*np.sin(omega_sb*t))
p_t = partial(p_t, omega_p, beta, omega_sb)


#Generate a spatial profile for the Rabi rate
Omega = 2*np.pi*1e6 #Peak Rate for laser
fwhm_z = 1e-3 #FWHM along molecular beam flight direction
fwhm_y = 5e-3 #FWHM along height of beam

#Define spatial profile of laser
E_peak = Omega 
laser_power = calculate_power_needed(Omega, 1, D_TlF = 1, fwhm_z = fwhm_z, fwhm_y = fwhm_y)
E_r = lambda r: multipassed_laser_E_field(r[2],r[1], power=laser_power, z0 = 3e-2/4,
                                              fwhm_z = fwhm_z, fwhm_y = fwhm_y, n_passes = 11,
                                              a = 0.002, t = 0.98)/E_peak
Omega_r = lambda r:  Omega*E_r(r)

In [ ]:
# t_test = np.linspace(0,T,10000)
# z_test = np.linspace(z0,z1,10000)

# E_test = np.array([E_r(np.array([0,0,z])) for z in z_test])
# Omega_test = np.array([Omega_r(np.array([0,0,z]))/(2*np.pi*1e6) for z in z_test])

# fig, ax = plt.subplots()
# ax.plot(z_test, E_test)

In [ ]:
#Define a LaserField object. Detuning set to zero
laser_field = OpticalField(p_t = p_t, ground_states = ground_states_laser_approx,
                          excited_states=excited_states_laser_approx, ground_main = ground_main_approx,
                          excited_main = excited_main_approx, Omega_r = Omega_r, detuning = 0)

In [ ]:
#The solver expects a list of LaserField objects so make one. Need a list to allow multiple lasers
laser_fields = [laser_field]

## Microwave fields
Define the microwave fields. Simular to LaserFields but use MicrowaveField objects. Don't need to explicitly define the ground and excited states, only the values of J for the levels that are coupled

In [ ]:
#Define Rabi rates
Omega0 = 2*np.pi*0
Omega1 = 2*np.pi*1e6
Omega2 = 2*np.pi*1e6

#Define polarizations
#Microwave polarization switching frequency
omega_p_mu = 0.1*Gamma

#Define polarization vectors as function of time
p0_t = lambda omega_p_mu, t: np.array([0, np.cos(omega_p_mu*t), np.sin(omega_p_mu*t)])
p0_t = partial(p0_t, omega_p_mu)

p1_t = lambda omega_p_mu, t: np.array([0, np.cos(omega_p_mu*t+np.pi/4), np.sin(omega_p_mu*t+np.pi/4)])
p1_t = partial(p1_t, omega_p_mu)

p2_t = lambda omega_p_mu, t: np.array([0, np.cos(omega_p_mu*t), np.sin(omega_p_mu*t)])
p2_t = partial(p2_t, omega_p_mu)


In [ ]:
#Define MicrowaveFields
mu0 = MicrowaveField(Omega_peak=Omega0, p_t = p1_t, Jg = 0, Je = 1)
mu1 = MicrowaveField(Omega_peak=Omega1, p_t = p1_t, Jg = 1, Je = 2)
mu2 = MicrowaveField(Omega_peak=Omega2, p_t = p2_t, Jg = 2, Je = 3)

#Make a list of the MicrowaveFields. mu0 not really needed if Omega0 = 0
microwave_fields = [mu0, mu1, mu2]

## Initial populations
Define states that are populated initially. A Boltzmann distribution is assumed, unless otherwise specified by passing a list of populations to integrator function

In [ ]:
# Define states that are populated initially
Js = [0,1,2,3]
states_pop = [1*CoupledBasisState(F,mF,F1,J,I_F,I_Tl, electronic_state='X', P = (-1)**J, Omega = 0)
              for J  in Js
              for F1 in ni_range(np.abs(J-I_Tl),J+I_Tl+1)
              for F in ni_range(np.abs(F1-I_F),F1+I_F+1)
              for mF in ni_range(-F, F+1)
             ]

## Run the integrator
Run the integrator using both the exponentiation and the RK45 methods

In [ ]:
%%time
t_array_exp, pop_results_exp = OBE_integrator(X_states = ground_states_approx, B_states = excited_states_approx,
                                       microwave_fields=microwave_fields, laser_fields = laser_fields,
                                       states_pop = states_pop, method = 'exp', verbose = False)

In [ ]:
%%time
t_array_RK45, pop_results_RK45 = OBE_integrator(X_states = ground_states_approx, B_states = excited_states_approx,
                                       microwave_fields=microwave_fields, laser_fields = laser_fields,
                                       states_pop = states_pop, method = 'RK45', verbose = False)

## Plotting

In [ ]:
#Plot populations in different J over time when using the exponentiation method
P0_triplet_exp = np.sum(pop_results_exp[1:4,:], axis = 0)
P0_singlet_exp = np.sum(pop_results_exp[0:1,:], axis = 0)
P1_exp = np.sum(pop_results_exp[4:16,:], axis = 0)
P2_exp = np.sum(pop_results_exp[16:36,:], axis = 0)
P3_exp = np.sum(pop_results_exp[36:64,:], axis = 0)
PB1_exp = np.sum(pop_results_exp[64:,:], axis = 0)

fig, ax = plt.subplots()
ax.plot(t_array_exp*1e6, P0_triplet_exp, label = 'X, J = 0, F = 1')
ax.plot(t_array_exp*1e6, P0_singlet_exp, label = 'X, J = 0, F = 0')
ax.plot(t_array_exp*1e6, P1_exp, label = 'X, J = 1')
ax.plot(t_array_exp*1e6, P2_exp, label = 'X, J = 2')
ax.plot(t_array_exp*1e6, P3_exp, label = 'X, J = 3')
ax.plot(t_array_exp*1e6, PB1_exp, label = 'B, J = 1')
ax.legend()
ax.set_xlabel("Time / us")
ax.set_ylabel("Population in state")

In [ ]:
#Plot populations in different J over time when using the RK45 method
P0_triplet_RK45 = np.sum(pop_results_RK45[1:4,:], axis = 0)
P0_singlet_RK45 = np.sum(pop_results_RK45[0:1,:], axis = 0)
P1_RK45 = np.sum(pop_results_RK45[4:16,:], axis = 0)
P2_RK45 = np.sum(pop_results_RK45[16:36,:], axis = 0)
P3_RK45 = np.sum(pop_results_RK45[36:64,:], axis = 0)
PB1_RK45 = np.sum(pop_results_RK45[64:,:], axis = 0)

fig, ax = plt.subplots()
ax.plot(t_array_RK45*1e6, P0_triplet_RK45, label = 'X, J = 0, F = 1')
ax.plot(t_array_RK45*1e6, P0_singlet_RK45, label = 'X, J = 0, F = 0')
ax.plot(t_array_RK45*1e6, P1_RK45, label = 'X, J = 1')
ax.plot(t_array_RK45*1e6, P2_RK45, label = 'X, J = 2')
ax.plot(t_array_RK45*1e6, P3_RK45, label = 'X, J = 3')
ax.plot(t_array_RK45*1e6, PB1_RK45, label = 'B, J = 1')
ax.legend()
ax.set_xlabel("Time / us")
ax.set_ylabel("Population in state")

In [ ]:
#Check that there are no negative populations
print(np.min(pop_results_exp))
print(np.min(pop_results_RK45))

In [ ]:
print(P0_singlet_exp[-1]/P0_triplet_exp[-1])
print(P0_singlet_RK45[-1]/P0_triplet_RK45[-1])

In [ ]:
#Print final populations in each state
print(P0_triplet_exp[-1])
print(P0_singlet_exp[-1])
print(P1_exp[-1])
print(P2_exp[-1])
print(P3_exp[-1])
print(PB1_exp[-1])

In [ ]:
#Print final populations in each state
print(P0_triplet_RK45[-1])
print(P0_singlet_RK45[-1])
print(P1_RK45[-1])
print(P2_RK45[-1])
print(P3_RK45[-1])
print(PB1_RK45[-1])